In [47]:
# !pip install --upgrade openai

In [ ]:
from openai import OpenAI

# create an OpenAI client using the API key
client = OpenAI(api_key=API_KEY)

In [ ]:
# import pymupdf4llm
# md_text = pymupdf4llm.to_markdown("6832_SercePehlevan_2020.pdf")

# import pathlib
# pathlib.Path("output.md").write_bytes(md_text.encode())

In [35]:
file = client.files.create(
    file=open("6832_SercePehlevan_2020.pdf", "rb"),
    purpose="user_data",
)

In [44]:
def generate_response_with_chatgpt(prompt):
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    { "type": "input_text", "text": prompt },
                    {
                        "type": "input_file",
                        "file_id": file.id,
                    }
                ]
            }
        ]
    )

    return response.output_text

In [ ]:
with open("domain_1_q_1_1.txt", "r", encoding="utf-8") as f:
    content = f.read()

answer = generate_response_with_chatgpt(content)

In [ ]:
from pathlib import Path

def process_domain_files(patterns=("domain_1*.txt", "domain_2*.txt")):
    """Read each domain text file and send its content to the model."""
    answers = {}
    for pattern in patterns:
        for path in sorted(Path('.').glob(pattern)):
            with path.open('r', encoding='utf-8') as f:
                content = f.read()
            response = generate_response_with_chatgpt(content)
            answers[path.name] = getattr(response, 'output_text', response)
    return answers

all_answers = process_domain_files()
for name, answer in all_answers.items():
    print(f"{name} -> {answer}")


In [7]:
import re
from collections import defaultdict
from pathlib import Path

# Map domain (variants via folder name) -> question code -> prompt question file path
prompt_question_files = defaultdict(dict)
pattern = re.compile(r"^domain_?(\d+)(?:_(.+))?$")
for prompt_path in Path('prompts').rglob('question_*.txt'):
    match = pattern.match(prompt_path.parent.name)
    if not match:
        continue
    domain_id, variant = match.groups()

    stem_parts = prompt_path.stem.split('_')
    qnum = stem_parts[1] if len(stem_parts) > 1 else 'unknown'
    question_code = f"{domain_id}.{qnum}"

    domain_key = f"domain_{domain_id}" if not variant else f"domain_{domain_id}_{variant}"
    prompt_question_files[domain_key][question_code] = str(prompt_path)

# Sort questions for stable output
prompt_question_files = {
    domain: {code: path for code, path in sorted(questions.items())}
    for domain, questions in sorted(prompt_question_files.items())
}

print('Prompt question files by domain (variants split via folder name):')
for domain, questions in prompt_question_files.items():
    print(domain)
    for code, f in questions.items():
        print(f" - {code}: {f}")


Prompt question files by domain (variants split via folder name):
domain_1
 - 1.1: prompts/domain_1/question_1.txt
 - 1.2: prompts/domain_1/question_2.txt
 - 1.3: prompts/domain_1/question_3.txt
domain_2_adhering
 - 2.1: prompts/domain_2_adhering/question_1.txt
 - 2.2: prompts/domain_2_adhering/question_2.txt
 - 2.3: prompts/domain_2_adhering/question_3.txt
 - 2.4: prompts/domain_2_adhering/question_4.txt
 - 2.5: prompts/domain_2_adhering/question_5.txt
 - 2.6: prompts/domain_2_adhering/question_6.txt
domain_2_assigment
 - 2.1: prompts/domain_2_assigment/question_1.txt
 - 2.2: prompts/domain_2_assigment/question_2.txt
 - 2.3: prompts/domain_2_assigment/question_3.txt
 - 2.4: prompts/domain_2_assigment/question_4.txt
 - 2.5: prompts/domain_2_assigment/question_5.txt
 - 2.6: prompts/domain_2_assigment/question_6.txt
 - 2.7: prompts/domain_2_assigment/question_7.txt
